In [175]:
import easyocr

In [178]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import object_detection
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
import cv2 
import numpy as np
from matplotlib import pyplot as plt

# Preprocessing

In [180]:
import math
import cv2
import numpy as np
from scipy import ndimage
from skimage.filters import threshold_local
from PIL import Image

In [181]:
img = cv2.imread('D:\\IA\\OCR\\Tensorflow\\workspace\\images\\test\\711.jpg')

In [182]:
def orientation_correction(img, save_image = False): 
    # GrayScale Conversion for the Canny Algorithm  
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    img_edges = cv2.Canny(img_gray, 100, 100, apertureSize=3)
    # Using Houghlines to detect lines
    lines = cv2.HoughLinesP(img_edges, 1, math.pi / 180.0, 100, minLineLength=100, maxLineGap=5)
    
    # Finding angle of lines in polar coordinates
    angles = []
    for x1, y1, x2, y2 in lines[0]:
        angle = math.degrees(math.atan2(y2 - y1, x2 - x1))
        angles.append(angle)
    
    # Getting the median angle
    median_angle = np.median(angles)
    
    # Rotating the image with this median angle
    img_rotated = ndimage.rotate(img, median_angle)
    
    return img_rotated


def bw_scanner(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    T = threshold_local(gray, 21, offset = 5, method = "gaussian")
    return (gray > T).astype("uint8") * 255


def noise_removal(image):
    import numpy as np
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    image = cv2.medianBlur(image, 3)
    return (image)
##
grayImage2 = cv2.cvtColor(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2RGB)
cv2.imwrite("D:\\IA\\OCR\\Tensorflow\\workspace\\images\\test\\output.jpg",finalImg)

True

# Model

In [159]:
PATH_TO_CFG = "D:\\IA\\OCR\\Tensorflow\\workspace\\models\\model2\\pipeline.config"

In [160]:
paths = {'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',"model2")}

In [161]:
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)

In [162]:
model_config = configs['model']

In [163]:
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

In [164]:
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-22')).expect_partial()

def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [165]:
PATH_TO_LABELS = "D:\\IA\\OCR\\Tensorflow\\workspace\\annotations\\label_map.pbtxt"

In [166]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,use_display_name=True)

In [167]:
IMAGE_PATH = "D:\\IA\\OCR\\Tensorflow\\workspace\\images\\test\\output.jpg"

In [168]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=10,
            min_score_thresh=.3,
            agnostic_mode=False)

cv2.imwrite('result.jpg',image_np_with_detections)

INFO:tensorflow:depth of additional conv before box predictor: 0


True

# EasyOCR

In [146]:
detection_threshold = 0.3

In [147]:
image = image_np_with_detections
width = image.shape[1]
height = image.shape[0]
scores = list(filter(lambda x: x> detection_threshold, detections['detection_scores']))
boxes = detections['detection_boxes'][:len(scores)]
classes = detections['detection_classes'][:len(scores)]

# EasyOCR

In [148]:
temp = []
for idx, box in enumerate(boxes):
    roi = box*[height, width, height, width]
    region = image[int(roi[0]):int(roi[2]),int(roi[1]):int(roi[3])]
    reader = easyocr.Reader(['ch_tra', 'en'])
    ocr_result = reader.readtext(region)
    for result in ocr_result:
        print(result[1])
        temp.append(result[1])

萬寧 mannings
Hastertard
凸舌(;巷咚)
475.70
4'6 05045952109033699 3
倍用卡
03-06-2018 17:38;5{
[195439396


In [149]:
print(temp)

['萬寧 mannings', 'Hastertard', '凸舌(;巷咚)', '475.70', "4'6 05045952109033699 3", '倍用卡', '03-06-2018 17:38;5{', '[195439396']


In [150]:
for i in temp:
    print(i)

萬寧 mannings
Hastertard
凸舌(;巷咚)
475.70
4'6 05045952109033699 3
倍用卡
03-06-2018 17:38;5{
[195439396


In [151]:
with open('EasyOCR_result.csv', 'w', encoding='utf-8-sig', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(temp)

# PaddleOCR

In [169]:
from paddleocr import PaddleOCR, draw_ocr
from matplotlib import pyplot as plt
import cv2
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from cv2 import resizeWindow
font=cv2.FONT_HERSHEY_SIMPLEX

In [170]:
# Setup model
#ocr_model = PaddleOCR(lang='chinese_cht')

In [171]:
ocr_model = PaddleOCR(use_angle_cls=True, lang="chinese_cht",use_gpu=True)

Namespace(benchmark=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\USER/.paddleocr/2.2.0.2\\ocr\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\USER/.paddleocr/2.2.0.2\\ocr\\det\\en\\en_ppocr_mobile_v2.0_det_infer', det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_polygon=True, e2e_pgnet_score_thresh=0.5, e2e_pgnet_valid_set='totaltext', enable_mkldnn=False, gpu_mem=500, help='==SUPPRESS==', image_dir=None, ir_optim=True, label_list=['0', '180'], lang

In [172]:
detection_threshold = 0.3
image = image_np_with_detections
width = image.shape[1]
height = image.shape[0]
scores = list(filter(lambda x: x> detection_threshold, detections['detection_scores']))
boxes = detections['detection_boxes'][:len(scores)]
classes = detections['detection_classes'][:len(scores)]

In [173]:
temp = []
for idx, box in enumerate(boxes):
    roi = box*[height, width, height, width]
    region = image[int(roi[0]):int(roi[2]),int(roi[1]):int(roi[3])]
    ocr_result = ocr_model.ocr(region)
    for result in ocr_result:
        print(result[1][0])
        temp.append(result[1][0])

[2021/08/28 02:18:32] root DEBUG: dt_boxes num : 1, elapse : 0.017951488494873047
[2021/08/28 02:18:32] root DEBUG: cls num  : 1, elapse : 0.010970115661621094
[2021/08/28 02:18:32] root DEBUG: rec_res num  : 1, elapse : 0.01595783233642578
8OK便利店
[2021/08/28 02:18:32] root DEBUG: dt_boxes num : 1, elapse : 0.011968135833740234
[2021/08/28 02:18:32] root DEBUG: cls num  : 1, elapse : 0.010970354080200195
[2021/08/28 02:18:32] root DEBUG: rec_res num  : 1, elapse : 0.014960289001464844
payment:84%
[2021/08/28 02:18:32] root DEBUG: dt_boxes num : 1, elapse : 0.009973287582397461
[2021/08/28 02:18:32] root DEBUG: cls num  : 1, elapse : 0.009973526000976562
[2021/08/28 02:18:32] root DEBUG: rec_res num  : 1, elapse : 0.013962745666503906
[2021/08/28 02:18:32] root DEBUG: dt_boxes num : 2, elapse : 0.009582042694091797
[2021/08/28 02:18:32] root DEBUG: cls num  : 2, elapse : 0.009970903396606445
[2021/08/28 02:18:32] root DEBUG: rec_res num  : 2, elapse : 0.014922380447387695


# Paddle_OCR

In [174]:
temp

['8OK便利店', 'payment:84%']

In [141]:
import csv

In [142]:
#f = open('D:\\IA\\OCR\\OCR_result', temp)

In [143]:
first_row = ['Brand_name', 'Total', 'Payment method', 'Data time',"recipt id"]

with open('Paddle_OCR_result.csv', 'w', encoding='utf-8-sig', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(temp)

In [144]:
import csv

# csv header
first_row = ['name', 'area', 'country_code2', 'country_code3']

# csv data
rows = [
    {'name': 'Albania',
    'area': 28748,
    'country_code2': 'AL',
    'country_code3': 'ALB'},
    {'name': 'Algeria',
    'area': 2381741,
    'country_code2': 'DZ',
    'country_code3': 'DZA'},
    {'name': 'American Samoa',
    'area': 199,
    'country_code2': 'AS',
    'country_code3': 'ASM'}
]

with open('countries.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)